In [50]:
import numpy as np
import math
from typing import Callable, TypeVar
from collections import Counter, deque, defaultdict
import itertools
from functools import cmp_to_key, cache
import regex as re
from intervaltree import Interval, IntervalTree
from concurrent.futures import ThreadPoolExecutor
from sortedcontainers import SortedDict
import z3

np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

processors = {
  'int_list': lambda x: [int(y) for y in x.split()],
  'int_string': lambda x: [int(y) for y in x]
}

def search(start, get_neighbors, end_condition=lambda _, __: False, dfs=True, permutations=False):
    q, visited = deque([(start, 0)]), {}
    while q:
        current, distance = q.popleft() if dfs else q.pop()
        if end_condition(current, distance):
            return visited, current
        if current in visited and not permutations:
            continue
        for node in get_neighbors(current, distance):
            q.append((node, distance+1))
        visited[current] = distance
    return visited, None

def sum_series(start, stop):
    n = (stop - start)
    sum = start + stop
    return n * sum // 2

def debug_array(arr, coords, p=True):
    arr = arr.copy()
    for i in coords:
        arr[*i] = 'X'
    if p:
        print(arr)
    return arr

def flatten(xss):
    return [x for xs in xss for x in xs]

def yx(n):
    return int(n.imag), int(n.real)
def icoord(y, x):
    return x+1j*y

In [74]:

def day1():
    loc1, loc2 = zip(*data(1, processors['int_list']))
    part1 = sum(abs(x[0]-x[1]) for x in zip(sorted(loc1), sorted(loc2)))
    counts = Counter(loc2)
    part2 = sum(x*counts[x] for x in loc1)
    return part1, part2

day1()

(1941353, 22539317)

In [75]:
def day2():
    def check_safe(report):
        ascending = sorted(report)
        diffs = np.diff(ascending)
        return max(diffs) <= 3 and min(diffs) >= 1 and (
            report == ascending or
            report == list(reversed(ascending))
        )

    def check_safe_damp(report):
        if check_safe(report):
            return 1, 1
        for damped in itertools.combinations(report, len(report)-1):
            if check_safe(list(damped)):
                return 0, 1
        return 0, 0

    reports = data(2, processors['int_list'])
    safe = np.array((0,0))
    for report in reports:
        safe += check_safe_damp(report)
    return safe

day2()

array([356, 413])

In [76]:
def day3():
    def mul_strings(s):
        x, y = s.split(',')
        return int(x)*int(y)

    instructions = ''.join(data(3))
    matches = list(re.finditer(r'mul\((\d+,\d+)\)', instructions))
    conds = list(re.finditer(r"don't\(\).+?do\(\)", instructions))
    donts = IntervalTree([Interval(*cond.span()) for cond in conds])
    result = sum([mul_strings(mul[1]) * (1 if not donts[mul.span()[0]] else 1j) for mul in matches])
    return int(result.real+result.imag), int(result.real)

day3()

(182780583, 90772405)

In [125]:
def day4():
    grid = np.array(data(4, lambda x: np.array(list(x))))
    ymax, xmax = grid.shape

    def find_target_occurences(target):
        occurences = set()

        def get_neighbors(current, distance):
            row, col = yx(current)
            target_letter = target[distance]
            if grid[row, col] != target_letter:
                return
            if distance == len(target)-1:
                total.add(current)
                return
            for v in [1, -1, 1j, -1j, 1+1j, 1-1j, -1+1j, -1-1j]:
                new = current + v
                y, x = yx(new)
                if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                    continue
                yield new
        
        for j in range(ymax):
            for i in range(xmax):
                total = set()
                coordinate = i+1j*j
                search(coordinate, get_neighbors)
                for end in total:
                    occurences.add((coordinate, end))
        return occurences

    def find_diags(hits, l):
        centers = Counter()
        for start, end in hits:
            distance = end-start
            if abs(distance.real) == l and abs(distance.imag) == l:
                center = start + distance/2
                centers[center] += 1
        return centers
    
    def find_straights(hits, target):
        rev = target[::-1]
        l = len(target)
        td = l-1
        for start, end in hits:
            j, i = yx(start)
            d = end-start
            if (
                (d.real == td and not d.imag and ''.join(grid[j, i:i+l]) == target)
                or (d.real == -td and not d.imag and ''.join(grid[j, i-td:i+1]) == rev)
                or (not d.real and d.imag == td and ''.join(grid[j:j+l, i]) == target)
                or (not d.real and d.imag == -td and ''.join(grid[j-td:j+1, i]) == rev)
            ):
                yield start

    target = 'XMAS'
    matches = find_target_occurences(target)
    part1 = sum(find_diags(matches, len(target)-1).values()) + len(list(find_straights(matches, target)))

    centers = find_diags(find_target_occurences(target[1:]), len(target)-2)
    part2 = sum([1 if centers[x] == 2 else 0 for x in centers])

    return (part1, part2)

day4()

(2599, 1948)

In [78]:
def day5():
    text = data(5)
    split = text.index('')
    lists = [tuple(map(int, x.split(','))) for x in text[split+1:]]

    parents = defaultdict(lambda: set())
    for x in text[:split]:
        parent, child = tuple(map(int, x.split('|')))
        parents[child].add(parent)

    def check_illegal(nums):
        illegal = set()
        for num in nums:
            if num in illegal:
                return True
            illegal.update(parents[num])

    def compare(a, b):
        if a in parents[b]:
            return 1
        elif b in parents[a]:
            return -1
        return -1 if a < b else 1

    part1, part2 = 0, 0
    for nums in lists:
        if not check_illegal(nums):
            part1 += nums[len(nums)//2]
        else:
            part2 += sorted(nums, key=cmp_to_key(compare))[len(nums)//2]
            
    return part1, part2

day5()

(6041, 4884)

In [73]:
def day6():
    grid = np.array(data(0, list))
    ymax, xmax = grid.shape
    start = np.argwhere(grid == '^')[0]
    grid[*start] = '.'
    turns = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    
    def run_guard(obstacle=(-1, -1)):
        directions, v = itertools.cycle(turns), turns[-1]
        y, x = int(start[0])-v[1], int(start[1])-v[0]
        visited, states = set(), set()
        while True:
            ny, nx = y+v[1], x + v[0]
            if (ny, nx, v) in states:
                return True, states
            elif (nx < 0 or ny < 0 or nx >= xmax or ny >= ymax):
                return False, visited
            elif grid[ny, nx] != '.' or (ny, nx) == obstacle:
                v = next(directions)
                continue
            y, x = ny, nx
            visited.add((y, x))
            states.add((y, x, v))

    _, original = run_guard()
    part1 = len(original)
    with ThreadPoolExecutor() as tpe:
        part2 = sum([r[0] for r in tpe.map(lambda x: run_guard(x), original)])
    return part1, part2

day6()

(41, 6)

In [11]:
def day7():
    equations = data(0, lambda x: [int(n) for n in re.split(r' |: ', x)])
    
    def check(n, acc, arr):
        if not arr:
            return n == acc
        elif acc > n:
            return False
        x, tail = arr[0], arr[1:]
        return (check(n, int(f'{acc}{x}'), tail) or
                check(n, acc*x, tail) or
                check(n, acc+x, tail))

    result = 0
    for eq in equations:
        if check(eq[0], eq[1], tuple(eq[2:])):
            result += eq[0]
    print(result)


day7()

11387


In [50]:
def day7():
    equations, flag = data(7, lambda x: [int(n) for n in re.split(r' |: ', x)]), False
    def check(n, acc, arr):
        if not arr:
            return n if n == acc else 0
        elif acc > n:
            return
        x, tail = arr[0], arr[1:]
        return ((flag and check(n, int(f'{acc}{x}'), tail)) or
                check(n, acc*x, tail) or
                check(n, acc+x, tail))
    part1, flag = sum([check(eq[0], eq[1], tuple(eq[2:])) for eq in equations]), True
    with ThreadPoolExecutor() as tpe:
        part2 = sum(tpe.map(lambda eq: check(eq[0], eq[1], tuple(eq[2:])), equations))
    return part1, part2

day7()

(7710205485870, 20928985450275)

In [72]:
def day8():
    grid = np.array(data(8, list))
    ymax, xmax = grid.shape
    def is_inside(y, x):
        return y >= 0 and y < ymax and x >= 0 and x < xmax
    def get_nodes(start, diff, sign=1):
        node = start.copy()
        while is_inside(*node):
            yield tuple(node)
            node -= sign*diff

    part1, part2 = set(), set()
    points = {i:np.argwhere(grid==i) for i in np.unique(grid) if i != '.'}
    for antennae in points:
        combos = itertools.combinations(points[antennae], 2)
        for combo in combos:
            diff = combo[1]-combo[0]
            for y, x in (combo[0]-diff, combo[1]+diff):
                if is_inside(y, x):
                    part1.add((y, x))
            for (y, x) in [*get_nodes(combo[0], diff), *get_nodes(combo[1], diff, -1)]:
                part2.add((y, x))

    return len(part1), len(part2)

day8()

(261, 898)

In [87]:
def day9():
    disk_map = data(9)[0]
    files = [int(x) for x in disk_map[0::2]]
    buffers = [int(x) for x in disk_map[1::2]]

    def part1(buffers):
        p = 0 # Current pointer location
        buffer = 0 # Current buffer space available
        fid = 0 # Current file number from left to right
        checksum = 0 # Result
        for n in range(len(files)-1, -1, -1):
            req = files[n]
            while req > buffer:
                # fill buffer with rightmost file
                checksum += sum_series(p, p+buffer)*n
                p += buffer
                req -= buffer
                # add current file to checksum
                fsize = files[fid]
                checksum += sum_series(p, p+fsize)*fid
                p += fsize
                fid += 1
                if fid >= n:
                    checksum += sum_series(p, p+req)*fid
                    return checksum
                # get next buffer
                buffer = next(buffers)
            checksum += sum_series(p, p+req)*n
            p += req
            buffer -= req

    def part2(buffers):
        p = 0
        starts = {}
        slots = SortedDict()
        for i, v in enumerate(files):
            starts[i] = p
            p += v
            try:
                buffer = next(buffers)
                if buffer:
                    slots[p] = buffer
                    p += buffer
            except StopIteration:
                continue

        def find_slot(size): # O(n)
            for index, value in slots.items():
                if value >= size:
                    return index
            return -1

        checksum = 0
        for n in range(len(files)-1, -1, -1):
            req = files[n]
            p = find_slot(req)
            if p != -1 and p < starts[n]:
                buffer = slots[p]
                del slots[p]
                if (rem := buffer-req):
                    slots[p+req] = rem
            else:
                p = starts[n]
            checksum += sum_series(p, p+req)*n
        return checksum
    
    return part1(iter(buffers)), part2(iter(buffers))

day9()

(6349606724455, 6376648986651)

In [126]:
def day10():
    grid = np.array(data(10, processors['int_string']))
    ymax, xmax = grid.shape
    starts = np.argwhere(grid==0)

    def get_neighbors(current, distance):
        for v in (1, -1, 1j, -1j):
            new = current + v
            y, x = yx(new)
            if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                continue
            if grid[y, x] == distance + 1:
                yield new

    def trail_end(current, distance):
        if distance == 9:
            trailheads.add(current)

    def rating_end(current, distance):
        if distance == 9:
            ratings.append(current)

    part1, part2 = 0, 0
    for y, x in starts:
        trailheads, ratings = set(), []
        search(x+1j*y, get_neighbors, trail_end)
        search(x+1j*y, get_neighbors, rating_end, permutations=True)
        part1 += len(trailheads)
        part2 += len(ratings)
    return part1, part2

day10()

(737, 1619)

In [7]:
def day11():
    stones = data(11, processors['int_list'])[0]

    def next_stones(s):
        if not s:
            return (1, )
        ss = str(s)
        half, rem = divmod(len(ss), 2)
        if not rem:
            return int(ss[:half + rem]), int(ss[half + rem:])
        return (s*2024, )
    
    @cache
    def run_stones(s, i):
        if not i:
            return [s]
        result = flatten([run_stones(stone, i-1) for stone in next_stones(s)])
        return result

    @cache
    def stone_count(s, i):
        if not i:
            return 1
        stones = run_stones(s, 1)
        result = 0
        for stone in stones:
            result += stone_count(stone, i-1)
        return result

    part1 = sum(stone_count(s, 25) for s in stones)
    part2 = sum(stone_count(s, 75) for s in stones)
    return part1, part2

day11()

(186424, 219838428124832)

In [129]:
def day12():
    grid = np.array(data(12, list))
    ymax, xmax = grid.shape
    
    def corner_count(n, y, x):
        size = len(n)
        if not size:
            return 4
        elif size == 1:
            return 2
        elif size == 2: 
            b, a = yx(n[0])
            j, i = yx(n[1])
            if j != b and i != a: # L shape
                return 1 if grid[j, a] == grid[b, i] else 2
        else:
            ys, xs = list(zip(*[yx(node) for node in n])) # Check diagonals
            bot, top, left, right = min(ys), max(ys), min(xs), max(xs)
            return len([i for i in [(bot, left), (bot, right), (top, left), (top, right)] if grid[i] != grid[y, x]])
        return 0

    def get_neighbors(current, _):
        target = yx(current)
        neighbors = []
        for v in (1, -1, 1j, -1j):
            new = current + v
            y, x = yx(new)
            if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                continue
            if grid[y, x] == grid[*target]:
                neighbors.append(new)
                perimeters[current] -= 1
        corners[current] = corner_count(neighbors, *target)
        return neighbors

    part1, part2 = 0, 0
    visited = set()
    perimeters, corners = defaultdict(lambda: 4), {}
    for j in range(ymax):
        for i in range(xmax):
            coordinate = i + 1j*j
            if coordinate in visited:
                continue
            region, _ = search(coordinate, get_neighbors)
            visited.update(region)
            part1 += len(region)*sum(perimeters[r] for r in region.keys())
            part2 += len(region)*sum(corners[r] for r in region.keys())
    return part1, part2

day12()

(1473276, 901100)

In [35]:
def day13():
    lines = data(13)
    groups = [lines[i:i+3] for i in range(0, len(lines), 4)]

    def parse_coords(text, prize=False):
        r = r'Prize: X=(\d+), Y=(\d+)' if prize else r'Button .: X\+(\d+), Y\+(\d+)'
        matches = re.findall(r, text)[0]
        return int(matches[0]), int(matches[1])

    def solve(a, b, prize):
        solver = z3.Optimize()
        x = z3.Int('x')
        y = z3.Int('y')
        solver.minimize(3*x+y)
        solver.add(a[0]*x + b[0]*y == prize[0])
        solver.add(a[1]*x + b[1]*y == prize[1])
        
        solver.check()
        result = solver.model()
        return 3*result[x].as_long() + result[y].as_long() if result else 0

    part1, part2 = 0, 0
    for group in groups:    
        a = parse_coords(group[0])
        b = parse_coords(group[1])
        prize = parse_coords(group[2], True)
        part1 += solve(a, b, prize)
        part2 += solve(a, b, (prize[0]+10000000000000, prize[1]+10000000000000))
    return part1, part2

day13()

(36758, 76358113886726)

In [52]:
def day14(wide, tall):
    robots = data(14, lambda line: [int(x) for x in re.findall(r'p=(\d+),(\d+) v=(-?\d+),(-?\d+)', line)[0]])
    
    def get_quadrants(xs):
        lr, tb = (wide-1)/2, (tall-1)/2
        quadrants = Counter()
        for y, x in xs:
            if x < lr and y < tb:
                quadrants[1] += 1
            elif x < lr and y > tb:
                quadrants[2] += 1
            elif x > lr and y < tb:
                quadrants[3] += 1
            elif x > lr and y > tb:
                quadrants[4] += 1
        return quadrants

    def run_robot(robot, n):
        x, y, vx, vy = robot
        fx, fy = (x+n*vx)%wide, (y+n*vy)%tall
        return fy, fx

    quadrants = get_quadrants([run_robot(robot, 100) for robot in robots])
    part1 = math.prod((quadrants[x] for x in range(1, 5)))

    def run_robots(n):
        ps = [run_robot(robot, n) for robot in robots]
        variance = np.var([p[0] for p in ps])+np.var([p[1] for p in ps])
        return variance, n

    _, n = min(run_robots(n) for n in range(wide*tall))
    ps = [run_robot(robot, n) for robot in robots]
    with open(f'outputs/{n}.txt', 'w+') as f:
        np.savetxt(f, debug_array(np.full((tall, wide), 'O'), ps, False), fmt='%s')
    return part1, n

day14(101, 103)

(231852216, 8159)

In [53]:
def day15():
    print(data(0))
day15()

['p=0,4 v=3,-3', 'p=6,3 v=-1,-3', 'p=10,3 v=-1,2', 'p=2,0 v=2,-1', 'p=0,0 v=1,3', 'p=3,0 v=-2,-2', 'p=7,6 v=-1,-3', 'p=3,0 v=-1,-2', 'p=9,3 v=2,3', 'p=7,3 v=-1,2', 'p=2,4 v=2,-3', 'p=9,5 v=-3,-3']
